In [1]:
# This code is a QA system that answers questions starting with Who, What, When, or Where.
# It uses various functions and regular expressions to extract information from the input question and find the corresponding answer

In [2]:
# !pip install wikipedia

In [3]:
#Importing necessary libraries
import wikipedia
import sys
import os
import lxml
from nltk.chunk import tree2conlltags
import re
from datetime import datetime
#Importing specific functions from NLTK library
from nltk import ne_chunk, pos_tag, word_tokenize

In [4]:
#Downloading required NLTK datasets
import nltk

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [5]:
#Defining a tokenizer function that applies named entity recognition (NER) on a given sentence
def tokenizer(sent):
    words = nltk.word_tokenize(sent) # tokenize sentence into words
    pos_tags = nltk.pos_tag(words) # assign part-of-speech tags to words
    tree = nltk.ne_chunk(pos_tags) # identify namled entities in the sentence
    conll_tags = nltk.tree2conlltags(tree) # convert named entities to CoNLL format
    return conll_tags

In [6]:
tokenizer('This is a New York City')

[('This', 'DT', 'O'),
 ('is', 'VBZ', 'O'),
 ('a', 'DT', 'O'),
 ('New', 'NNP', 'B-GPE'),
 ('York', 'NNP', 'I-GPE'),
 ('City', 'NNP', 'I-GPE')]

In [7]:
# [\d]+: one or more digits (0-9)
# \s: a whitespace character (space, tab, newline, etc.)
# (\w)+: one or more characters (letters, digits, or underscore)
# \s: another whitespace character
# [\d]+: one ore more digits again

In [8]:
"The function get_question takes a string s as input and return a matching question based on the provided rules."
"The regular expressions in the rules check for certain patterns in the string and return the corresponding groups."

def get_question(s):
    rules = [[r'Where (Is|Was) (.+)', ["{0}"]],
             [r'Who (Is|Was) (.+)', ["{0}"]],
             [r'What (Is|Was) (.+) Age', ["{0}"]],
             [r'What (Is|Was) (.+)', ["{0}"]],
             [r'When (Is|Was) (.+) Born', ["{0}"]],
             [r'When (Is|Was) (.+) Birthday', ["{0}"]],
             [r'When (Is|Was) (.+)', ["{0}"]],
            ]
    for r, c in rules:
            m = re.match(r, s.rstrip('.!?'))
            if m:
                return m.groups()[1]
    return ""

In [9]:
get_question("When was George Washington born?".title())

'George Washington'

In [10]:
get_question("Where is the capital of canada?".title())

'The Capital Of Canada'

In [11]:
"""
The function get_ans_wiki takes a query string and attempts to fetch a summary from Wikipedia
using the wikipedia.summary() function. It handles exceptions if the summary is not found.
"""
def get_ans_wiki(query):
    try:
        return wikipedia.summary(query, sentences=1)
    except wikipedia.exceptions.DisambiguationError as e:
        return wikipedia.summary(e.options[0], sentences=1)
    except wikipedia.exceptions.PageError:
        return ""
    except Exception:
        return ""

In [12]:
get_ans_wiki('The Capital of Canada')

'Ottawa ( , ; Canadian French: [ɔtawɑ]) is the capital city of Canada.'

In [13]:
"""
The function check_digits takes a text input and checks for specific patterns using
regular expressions. It returns the matching groups if a pattern is found.
"""
def check_digits(text):
    return re.findall(r'\d+', text)

In [14]:
get_question('When is the Freedom day?'.title())

'The Freedom Day'

In [15]:
tokenizer('When was George Washington born?'.title())

[('When', 'WRB', 'O'),
 ('Was', 'NNP', 'B-PERSON'),
 ('George', 'NNP', 'I-PERSON'),
 ('Washington', 'NNP', 'I-PERSON'),
 ('Born', 'NNP', 'I-PERSON'),
 ('?', '.', 'O')]

In [16]:
get_ans_wiki(get_question('When was George Washington born?'.title()))

'George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797.'

In [17]:
check_digits(get_ans_wiki(get_question('When was George Washington born?'.title())))

['22', '1732', '14', '1799', '1789', '1797']

In [18]:
"""
The transform() function takes two argumlents: 'u' and 'q'.
It removes any punctuation from the of of 'u' and then applies different regular expressions to
transform the question. If any transformation is successful, it returns the transformed question
with 'q' concatenated and a period added at the end.
"""
def transform(u, q):
    u = u.rstrip('.!?')
    ur = re.sub(r'When (Is|Was) (.+) Born', r'\2 Was Born On ', u)
    if ur != u:
        return ur + str(q) + '.'
    
    ur = re.sub(r'When (Is|Was) (.+) Birthday', r'\2 Birthday Is On ', u)
    if ur != u:
        ur = ur + q
        ur = re.sub(r'\d{4}', '', ur)
        return ur + '.'
    
    ur = re.sub(r'What (Is|Was) (.+) Age', r'\2 Age Is ', u)
    if ur != u:
        return ur + str(q) + '.'

In [19]:
transform("when is Trump Born?".title(), "1994")

'Trump Was Born On 1994.'

In [20]:
transform("When Was George Washington Born?".title(), 1732)

'George Washington Was Born On 1732.'

In [ ]:
"""
The main() function is the main entry point of the program.
It initializes a log file, prompts the user for input, and processes the input to find the answer.
It uses the tokenizer() and get_question() functions to prepare the input for further processing.
Based on the type of question ('Where', 'What', 'Who', 'When', or 'Age'), it retrieves
info from Wikipedia using the get_ans_wiki() function. For 'Who' and 'Age' questions,
it also uses the check_digits() function to extract relevent info.
Finally, it prints the answer to the user and writes it to the log file.
"""

def main():
    try:
        log = open('mylogfile.txt', "w+")
        print("*** This is a QA system by Group 4. It will try to answer questions that start with Who, What, When or Where. Enter 'exit' to leave the program.")
        counter = 1
        while True:
            s = input()
            e = s
            
            if s == "exit":
                print("\nThank you! Goodbye.")
                break
            
            log.write(str(counter) + "Ques) " + s + "\n")
            flag = False
            final = ""
            wiki = ""
            
            if s != "" and s is not None:
                
                s = s.title()
                n = tokenizer(s)
                r = get_question(s)
                
                if r!= "":
                    if ("Where" in s or "What" in s):
                        
                        wiki = get_ans_wiki(r)
                        match = re.search(r'\d[4]', wiki)
                        
                        if match is not None:
                            wiki = ""
                    
                    if ('I-PERSON' or 'B-PERSON') in n[2]:
                        flag = True
                        
                    if "Who" in s and flag:
                        wiki = get_ans_wiki(r)
                    
                    if ("When" in s) and flag:
                        
                        wiki = get_ans_wiki(r)
                        result = check_digits(wiki)
                        
                        if result != "":
                            for dd in result:
                                if(len(dd)==4):
                                    match = dd
                                    break
                                    
                            appends = transform(s, match)
                            if appends is not None:
                                final = appends
                        else:
                            final = ""
                    else:
                        final = get_ans_wiki(r)
                else:
                    final = ""
            else:
                print("Please ask a valid question!")
            if final == "":
                log.write(str(counter) + "A) Answer not found.\n\n")
                print("I am sorry, I don't know the answer.\n")
                
            else:
                try:
                    log.write(str(counter) + "A) " + final + "\n\n")
                    print(final + "\n")
                except Exception as GeneralException:
                    log.write(str(counter) + "A) Answer not found.\n\n")
                    
            counter = counter + 1
            print("Please ask another question or type 'exit' to exit from the program:\n")
    except Exception as GeneralException:
        print(GeneralException)
    finally:
        log.close()
        
main()

*** This is a QA system by Group 4. It will try to answer questions that start with Who, What, When or Where. Enter 'exit' to leave the program.
